---
title: "The Newton Method and its Applications"
page-layout: article
jupyter: julia-1.11
---

In [ ]:
#| echo: false
include("activate.jl")

The Newton method, also known as Newton-Raphson, is a powerful iterative technique widely used for solving nonlinear equations and optimization problems. In the context of nonlinear equations, the method seeks a root of a function $f(x) = 0$ by using the derivative information to iteratively refine an initial guess. For optimization, the method can be adapted to minimize a function by seeking the stationary points where its gradient vanishes. While Newton's method is highly efficient, requiring quadratic convergence under certain conditions, it also relies on the computation of derivatives, making it less suitable for some problems.

To build up to Newton's method, we first introduce the [bisection method](https://en.wikipedia.org/wiki/Bisection_method). Unlike Newton's method, the bisection method does not require derivatives, making it an excellent starting point for understanding root-finding techniques. 

## The Bisection Method

The bisection method is an iterative numerical technique for finding a root of a continuous function $f(x)$ on a closed interval $[a, b]$. The method assumes that $f(a)$ and $f(b)$ have opposite signs, which guarantees, by the Intermediate Value Theorem, that there is at least one root in $[a, b]$. The procedure works by repeatedly halving the interval and selecting the subinterval where the sign change occurs. 

### Exercise: Implement the Bisection Method

1. **Objective**: Implement the bisection method in Julia and analyze its behavior.

2. **Function Signature**: Define a function `bisection(f, a, b; tol=1e-12, max_iter=100)` that takes:
   - A continuous function `f`.
   - The interval bounds `a` and `b`.
   - An optional tolerance `tol` for stopping criteria.
   - An optional maximum number of iterations `max_iter`.

The function should return:

   - An approximation of the root.
   - The number of iterations performed.

3. **Example Problem**: Use the bisection method to find the root of $f(x) = \cos(x) - x$ on the interval $[0, 1]$.

4. **Analysis**:
   - Record the approximate root at each iteration.
   - Calculate the absolute error $|x_k - x^*|$ where $x^* = 0.7390851332151607$ is the true root.
   - Plot the error on a logarithmic scale and determine the [order of convergence](https://en.wikipedia.org/wiki/Rate_of_convergence).

5. **Discussion**:
   - Observe the behavior of the error as the iterations progress.
   - Conclude that the bisection method has a **linear convergence order**.

::: {.callout-tip collapse="true" icon=false}
### Hint for Implementation

- Use a loop to halve the interval and check the signs of $f(a)$, $f(b)$, and $f(\text{midpoint})$.
- Stop the iterations when $|b - a| / 2 \leq \text{tol}$ or the maximum number of iterations is reached.
- Handle cases where the initial interval does not satisfy the conditions for the method.
:::

::: {.callout-caution collapse="true" icon=false}
### Correction for the Bisection Method


In [ ]:
function bisection(f, a, b; tol=1e-12, max_iter=100)
    # Check if the initial interval is valid
    if f(a) * f(b) > 0
        error("The function must have opposite signs at the endpoints a and b.")
    end
    
    # Initialize variables
    mid = (a + b) / 2
    iter_count = 0
    
    while (b - a) / 2 > tol && iter_count < max_iter
        iter_count += 1
        mid = (a + b) / 2
        
        # Check if the midpoint is a root
        if isapprox(f(mid), 0, atol=tol)
            return mid, iter_count
        elseif f(a) * f(mid) < 0
            b = mid  # Root is in [a, mid]
        else
            a = mid  # Root is in [mid, b]
        end
    end
    
    # Return the midpoint as the approximate root
    return mid, iter_count
end

# Example usage
f(x) = cos(x) - x
root, iterations = bisection(f, 0, 1)
println("Approximate root: $root")
println("Evaluated function at the root: $(f(root))")
println("Number of iterations: $iterations", "\n")

# Exact root for comparison
exact_root = 0.7390851332151607
println("Exact root: $exact_root")
println("Evaluated function at the exact root: $(f(exact_root))")
println("Absolute error: $(abs(root - exact_root))")
println("Relative error: $(abs(root - exact_root) / exact_root)")

:::

::: {.callout-caution collapse="true" icon=false}
### Correction for Analysis

To analyze the convergence of the bisection method for $f(x) = \cos(x) - x$ on the interval $[0, 1]$, we compute the approximate root at each iteration and calculate the absolute error. Using the exact root $x^* \approx 0.7390851332151607$, we determine the error $|x_k - x^*|$. We also plot the error on a logarithmic scale to observe the convergence behavior.

Below is the Julia code for the analysis:


In [ ]:
using Plots

function bisection_analysis(f, a, b; tol=1e-12, max_iter=100, true_root=nothing)
    if f(a) * f(b) > 0
        error("The function must have opposite signs at the endpoints a and b.")
    end

    midpoints = []
    errors = []
    iter_count = 0
    mid = (a + b) / 2
    
    while (b - a) / 2 > tol && iter_count < max_iter
        iter_count += 1
        mid = (a + b) / 2
        push!(midpoints, mid)
        
        if !isnothing(true_root)
            push!(errors, abs(mid - true_root))
        end

        if isapprox(f(mid), 0, atol=tol)
            break
        elseif f(a) * f(mid) < 0
            b = mid
        else
            a = mid
        end
    end
    
    return mid, iter_count, midpoints, errors
end

# Exact root for reference
true_root = 0.7390851332151607
f(x) = cos(x) - x
root, iterations, midpoints, errors = bisection_analysis(f, 0, 1, true_root=true_root)

# Print the approximate root and error
println("Approximate root: $root")
println("Number of iterations: $iterations")
println("Final error: $(errors[end])")

# Plot the errors on a logarithmic scale
plt = plot(1:iterations, errors, yscale=:log10, xlabel="Iteration", ylabel="Absolute Error", 
    title="Convergence of the Bisection Method", label="Computed Error")

# Compute by linear regression the convergence rate
using Polynomials
line = fit(1:iterations, log10.(errors), 1)
println("Convergence rate: $(10^(line.coeffs[2]))")

# add the linear regression line to the plot
plot!(plt, 1:iterations, 10 .^ line.(1:iterations), label="Linear Regression")

:::


## The Newton Method

The **Newton Method**, or **Newton-Raphson Method**, is an iterative numerical technique for finding roots of a nonlinear equation $f(x) = 0$. Unlike the bisection method, Newton's method leverages the derivative of $f(x)$ to achieve a much faster rate of convergence under suitable conditions. Specifically, it exhibits **quadratic convergence**, meaning the number of accurate decimal places roughly doubles at each iteration once close to the root.

### Newton's Iteration Formula

Given an initial guess $x_0$, the Newton iteration is defined as:

$$
x_{k+1} = x_k - \frac{f(x_k)}{f'(x_k)},
$$

where $f'(x_k)$ is the derivative of $f(x)$ evaluated at $x_k$. The method relies on the approximation of $f(x)$ by its tangent line at $x_k$. The next iterate $x_{k+1}$ is where this tangent line intersects the $x$-axis.


In [ ]:
using LinearAlgebra
using Plots

# Function illustrating the Newton method for solving nonlinear equations
function illustration_newton(f::Function, df::Function, x0::Real, x_sol::Real, max_iter::Integer,
    a::Real, b::Real, 
    f_xlim::Tuple{Real, Real}, f_ylim::Tuple{Real, Real},
    g_xlim::Tuple{Real, Real}, g_ylim::Tuple{Real, Real},
    filename::String)

    # Initial values
    xk = x0
    x_sol = [x_sol]  # Exact solution

    # Store the iterations of the Newton method
    xs = [xk]
    errors = []

    # Newton's method iteration
    for iter in 1:max_iter
        # Update the Newton step
        xk = xk - f(xk) / df(xk)
        push!(xs, xk)
        push!(errors, abs(xk - x_sol))  # Error at each step

        # Check for convergence
        if abs(f(xk)) < 1e-12
            break
        end
    end

    N = length(xs)
    
    # Generate a range for plotting the function and its tangent lines
    x_span = range(a, b, length=101)

    # Define the function and its gradient for plotting
    pf_style = (xlim=f_xlim, ylim=f_ylim, linewidth=2, label="f(x)")
    pf = plot(x_span, f, pf_style...)

    # Plotting the gradient of the function
    pg_style = (xlim=g_xlim, ylim=g_ylim, linewidth=2, label="f'(x)")
    pg = plot(x_span, df, pg_style...)

    # Compute the errors
    erreur_style = (yaxis=:log, xlim=(-0.1, N-1+0.1), ylim=(minimum(errors)/2, 1), linewidth=2, label="Error")
    pe = plot(0:1:N-1, errors, erreur_style...)

    # Set up plot styles
    x_style = (color=:black, seriestype=:scatter, markersize=3, markerstrokewidth=0, label="")
    s_style = (color=:red, seriestype=:scatter, markersize=3, markerstrokewidth=0, label="")
    a_style = (color=:black, linestyle=:dash, label="")
    T_style = (color=:green, z_order=:back, label="")
    text_style = (annotationcolor=:black, annotationfontsize=10, annotationhalign=:center)

    # Plot the Newton steps and the tangents
    N_substeps = 3
    ps = plot(pf, pg, pe, layout=(1, 3), size=(900, 400))

    # Animation loop for visualizing the Newton method
    anim = @animate for k = 0:1:(N_substeps * N)
        if k == 0
            plot!(ps[1], x_sol, [f(x_sol)], s_style...)  # Exact solution point
            plot!(ps[2], x_sol, [0], s_style...)  # Gradient line at the solution
        else
            i, j = divrem(k-1, N_substeps)
            x = xs[i + 1]
            fx = f(x)
            dfx = df(x)

            # Draw point on the function
            if j == 0
                plot!(ps[1], x, [f(x)], x_style...)  # Function curve point
                plot!(ps[2], x, [df(x)], x_style...)  # Gradient curve point

                # Annotate the iteration number
                x_str = "x" * string(i)
                plot!(ps[1], annotation=((x, f(x), x_str)); annotationvalign=:top, text_style...)
                plot!(ps[2], annotation=((x, 0, x_str)); annotationvalign=:bottom, text_style...)
                
                # Plot tangent line at current point
                tangent(x) = f(x) + (x - xs[i]) * df(xs[i])
                plot!(ps[1], x_span, tangent.(x_span), a_style...)  # Tangent line

                # Plot error evolution
                plot!(ps[3], [i], [errors[i+1]], x_style...)
            elseif j == 1
                plot!(ps[1], x, [f(x)], x_style...)  # Function curve point
                plot!(ps[2], x, [df(x)], x_style...)  # Gradient curve point
            else
                # Plot the quadratic approximation
                if i + 1 < N
                    plot!(ps[1], x_span, f(xs[i]) + (x_span .- xs[i]) * df(xs[i]), T_style...)
                end
            end
        end
    end

    plot!(ps)
    gif(anim, filename, fps=1)
end

f(x) = cos(x) - x
df(x) = -sin(x) - 1
illustration_newton(f, df, 0.5, 0.7390851332151607, 10, -2, 2, (-2, 2), (-1, 1), (-2, 2), (-1, 1), "newton_method.gif")

### Exercise: Apply Newton's Method to Find a Root

1. **Objective**: Use the Newton method to find the root of $f(x) = \cos(x) - x$, starting with the initial guess $x_0 = 0$.

2. **Function Signature**: Define a function `newton(f, df, x0; tol=1e-12, max_iter=100)` that takes:
   - A continuous function `f`.
   - Its derivative `df`.
   - The initial guess `x0`.
   - An optional tolerance `tol` for stopping criteria.
   - An optional maximum number of iterations `max_iter`.

The function should return:

   - The approximate root.
   - The number of iterations performed.

3. **Example Problem**: Use the Newton method to find the root of $f(x) = \cos(x) - x$ with $f'(x) = -\sin(x) - 1$ starting at $x_0 = 0$.

4. **Analysis**:
   - Record the approximate root at each iteration.
   - Calculate the absolute error $|x_k - x^*|$ where $x^* \approx 0.7390851332151607$ is the true root.
   - Plot the error on a logarithmic scale and determine the **order of convergence**.

::: {.callout-tip collapse="true" icon=false}
### Hint for Implementation

- Use the Newton iteration formula to update the guess at each iteration.
- Stop the iterations when $|x_{k+1} - x_k| \leq \text{tol}$ or the maximum number of iterations is reached.
:::

::: {.callout-caution collapse="true" icon=false}
### Correction for the Newton Method


In [ ]:
function newton(f, df, x0; tol=1e-12, max_iter=100)
    xk = x0
    iter_count = 0

    while iter_count < max_iter
        iter_count += 1
        x_next = xk - f(xk) / df(xk)

        # Check if the difference is within the tolerance
        if abs(x_next - xk) < tol
            return x_next, iter_count
        end

        xk = x_next
    end

    return xk, iter_count
end

# Example usage
f(x) = cos(x) - x
df(x) = -sin(x) - 1
x0 = 0
root, iterations = newton(f, df, x0)

println("Approximate root: $root")
println("Evaluated function at the root: $(f(root))")
println("Number of iterations: $iterations", "\n")

# Exact root for comparison
exact_root = 0.7390851332151607
println("Exact root: $exact_root")
println("Evaluated function at the exact root: $(f(exact_root))")
println("Absolute error: $(abs(root - exact_root))")
println("Relative error: $(abs(root - exact_root) / exact_root)")

:::

::: {.callout-caution collapse="true" icon=false}
### Correction for Analysis

To analyze the convergence of the Newton method for $f(x) = \cos(x) - x$, we compute the approximate root at each iteration and calculate the absolute error. Using the exact root $x^* \approx 0.7390851332151607$, we determine the error $|x_k - x^*|$. We also plot the error on a logarithmic scale to observe the convergence behavior.

Below is the Julia code for the analysis:


In [ ]:
using Plots
using Polynomials

function newton_analysis(f, df, x0; tol=1e-12, max_iter=100, true_root=nothing)
    xk = x0
    iter_count = 0
    approximations = []
    errors = []

    while iter_count < max_iter
        iter_count += 1
        x_next = xk - f(xk) / df(xk)
        push!(approximations, x_next)

        if !isnothing(true_root)
            push!(errors, abs(x_next - true_root))
        end

        if abs(x_next - xk) < tol
            break
        end

        xk = x_next
    end

    return approximations, errors
end

# Exact root for reference
true_root = 0.7390851332151607
f(x) = cos(x) - x
df(x) = -sin(x) - 1
x0 = 0
approximations, errors = newton_analysis(f, df, x0, true_root=true_root)

# Exclude zero values for quadratic fitting
non_zero_errors = filter(e -> e > 0, errors)  # Only non-zero errors
indices_non_zero = findall(e -> e > 0, errors)  # Indices of non-zero errors

if length(non_zero_errors) > 2
    # Fit a quadratic polynomial to the log of non-zero errors
    p = fit(indices_non_zero, log10.(non_zero_errors), 2)  # Fit quadratic to log10 of errors
    println("Quadratic fit: $p")

    # Replace zero errors with the values predicted by the quadratic fit
    for i in 1:length(errors)
        if errors[i] == 0
            errors[i] = 10^p(i)  # Replace with the fitted value at the iteration
        end
    end
end

# Print the approximate root and error
println("Approximate root: $(approximations[end])")
println("Final error: $(errors[end])")

# Create the plot
plt = plot(1:length(errors), errors, yscale=:log10, xlabel="Iteration", ylabel="Absolute Error", 
    title="Convergence of the Newton Method", label="Computed Error")

# Add the fitted quadratic to the plot
iterations = 1:length(errors)
fitted_values = 10 .^ p.(iterations)
plot!(plt, iterations, fitted_values, label="Quadratic Fit", linestyle=:dash)

# Show the plot
display(plt)

:::